# Tables

In [39]:
table_prefix = f'google_ads_'
table_suffix = 'postfix'
tables = [
    'campaign_asset_performance',
    'placement_performance',
    'campaign_time_performance',
    'landing_page_performance',
    'ad_group_details',
    'ad_group_ads_performance',
    'geo_targets_constants_details',
    'ad_group_asset_performance',
    'geo_targets_details',
    'campaign_details',
    'campaign_conversion_goals_details',
    'campaign_crm_performance',
    'expanded_landing_page_performance',
    'combined_audience_details',
    'campaign_performance',
    'search_term_analysis_performance',
    'audience_performance',
    'detailed_demographics_details',
    'asset_group_assets_details',
    'campaign_device_performance',
    'gender_performance',
    'ad_group_performance',
    'ad_group_ads_details',
    'custom_audience_details',
    'ad_group_targeting_criteria_details',
    'audience_details',
    'parental_status_performance',
    'campaign_audience_performance',
    'income_range_performance',
    'ad_group_ad_asset_performance',
    'age_range_performance',
    'campaign_targeting_criteria_details',
    'customer_details',
    'keyword_performance'
    ]

table_name = tables[0]

In [40]:
f"{table_prefix}{tables[-1]}{table_suffix}"

'google_ads_keyword_performancepostfix'

# Sample Queries

In [ ]:
[
  {
    "query": "",
    "sql": "",
    "tables": [tables[0], tables[-1]],
    # Not considering columns for now.
    # "columns": [f"{tables[0]}.{columns[0][4]}", f"{tables[-1]}.{columns[-1][6]}"]
  },

  
]

other loaded in .json file

# RAG

In [ ]:
!pip install sentence-transformers faiss-cpu


In [ ]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Load your formatted data
with open("RaTsql/notebooks/formatted_data.json") as f:
    data = json.load(f)

# Step 1: Extract queries and metadata
queries = [item['query'] for item in data]
metadata = [ {k: item[k] for k in item if k != "query"} for item in data]

# Step 2: Embed queries
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")  # Small & fast
embeddings = model.encode(queries, convert_to_numpy=True)

# Step 3: Create FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)


In [43]:

# Step 4: Simple RAG retrieval function
def retrieve_similar(input_query, k=5):
    query_embedding = model.encode([input_query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, k)

    results = []
    for idx, dist in zip(indices[0], distances[0]):
        results.append({
            "retrieved_query": queries[idx],
            "similarity_score": float(1 / (1 + dist)),  # optional similarity score
            **metadata[idx]
        })
    return results

### FANCY OUTPUT

In [44]:
from rich.console import Console
from rich.markdown import Markdown
from rich.panel import Panel
from rich.syntax import Syntax

console = Console()

In [45]:
input_q = "Over the past 7 days - Which campaign had the highest conversion rate?"
top_k = retrieve_similar(input_q, k=5)

In [46]:
for i, item in enumerate(top_k, 1):
    console.rule(f"[bold green]Match #{i}[/bold green]", style="green")

    # 💠 Similarity Score (shown on top)
    score = round(item.get("similarity_score", 0.0) * 100, 2)
    console.print(f"[bold blue]🔗 Similarity Score:[/bold blue] [bold magenta]{score}[/bold magenta]%")

    # 📝 Retrieved Query
    console.print(Panel.fit(
        f"[bold cyan]{item['retrieved_query']}[/bold cyan]",
        title="🔍 Retrieved Query",
        border_style="cyan"
    ))

    # 📦 Tables
    table_str = ", ".join(item.get("tables", [])) or "None"
    console.print(Panel.fit(
        f"[bold yellow]{table_str}[/bold yellow]",
        title="📦 Tables",
        border_style="yellow"
    ))

    # 🧱 Columns
    column_str = ", ".join(item.get("columns", [])) or "None"
    console.print(Panel.fit(
        f"[bold white]{column_str}[/bold white]",
        title="📊 Columns",
        border_style="blue"
    ))

    # 🧠 SQL Query (syntax highlighted)
    sql_highlight = Syntax(item["sql"], "sql", theme="monokai", line_numbers=False)
    console.print(Panel(sql_highlight, title="🧠 SQL Query", border_style="magenta"))

    console.print("\n")

──────────────────────────────────────────────────── Match #1 ─────────────────────────────────────────────────────

🔗 Similarity Score: 92.4%

╭────────────────────────── 🔍 Retrieved Query ──────────────────────────╮
│ Over the past 14 days - Which campaign had the lowest conversion rate? │
╰────────────────────────────────────────────────────────────────────────╯

╭────────────── 📦 Tables ───────────────╮
│ campaign_details, campaign_performance │
╰────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📊 Columns ───────────────────────────────────────────────────╮
│ metrics_interactions, campaign_id, metrics_conversions, customer_id, segments_date, campaign_name,              │
│ conversion_rate                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

🔗 Similarity Score: 56.23%

╭──────────────────────── 🔍 Retrieved Query ─────────────────────────╮
│ Which ad group had the maximum cost per conversion on 2 June 2025 ? │
╰─────────────────────────────────────────────────────────────────────╯

╭────────────── 📦 Tables ───────────────╮
│ ad_group_details, ad_group_performance │
╰────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📊 Columns ───────────────────────────────────────────────────╮
│ metrics_conversions, cost_per_conversion, metrics_cost_micros, customer_id, ad_group_name, segments_date,       │
│ ad_group_id                                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── 🧠 SQL Query ──────────────────────────────────────────────────╮
│ SELECT gag.ad_group_name,                                                                                       │
│        SAFE_DIVIDE(SAFE_DIVIDE(SUM(ga.metrics_cost_micros), 1000000), SUM(ga.metrics_conversions)) AS cost_per_ │
│ FROM projectx-pipeline.dataset.google_ads_ad_group_performancepostfix ga                                        │
│ JOIN projectx-pipeline.dataset.google_ads_ad_group_detailspostfix gag ON ga.ad_group_id = gag.ad_group_id       │
│ WHERE ga.customer_id = 3424742100                                                                               │
│ AND ga.segments_date = '2025-06-02'                                                                             │
│ GROUP BY gag.ad_group_name                                                                                      │
│ ORDER BY cost_per_conversion DESC                                                                               │
│ LIMIT 1                                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Pruned Table Schema : Similar table/columns combine 

In [47]:
from collections import defaultdict
from rich.console import Console
from rich.pretty import Pretty
import json

# Constants
TABLE_PREFIX = "google_ads_"
TABLE_SUFFIX = "postfix"

# Group: table -> set of columns
table_columns = defaultdict(set)
examples = []

# Collect pruned schema + sql/query pairs
for item in top_k:
    tables = item.get("tables", [])
    columns = item.get("columns", [])
    query = item.get("retrieved_query", "")
    sql = item.get("sql", "")

    examples.append({
        "query": query,
        "sql": sql
    })

    for table in tables:
        for col in columns:
            table_columns[table].add(col)

# Convert to final format
pruned_schema = {
    f"{TABLE_PREFIX}{table}{TABLE_SUFFIX}": sorted(list(cols))
    for table, cols in table_columns.items()
}

# Output bundle
output_bundle = {
    "pruned_schema": pruned_schema,
    "examples": examples
}

# ✅ Pretty print
console = Console()
console.rule("[bold green]🧩 Pruned Table Schema[/bold green]")
console.print(Pretty(pruned_schema, expand_all=True))

console.rule("[bold cyan]💡 Retrieved Examples[/bold cyan]")
for i, ex in enumerate(examples, 1):
    console.print(f"[bold blue]Example #{i}[/bold blue]")
    console.print(f"[bold]Query:[/bold] {ex['query']}")
    console.print(f"[bold]SQL:[/bold] {ex['sql']}\n")

# ✅ Save to file if needed
with open("rag_output.json", "w") as f:
    json.dump(output_bundle, f, indent=2)


───────────────────────────────────────────── 🧩 Pruned Table Schema ──────────────────────────────────────────────

{
    'google_ads_campaign_detailspostfix': [
        'CAC',
        'campaign_id',
        'campaign_name',
        'conversion_rate',
        'customer_id',
        'metrics_conversions',
        'metrics_impressions',
        'metrics_interactions',
        'new_customers',
        'segments_date',
        'spend',
        'total_impressions'
    ],
    'google_ads_campaign_performancepostfix': [
        'campaign_id',
        'campaign_name',
        'conversion_rate',
        'customer_id',
        'metrics_conversions',
        'metrics_impressions',
        'metrics_interactions',
        'segments_date',
        'total_impressions'
    ],
    'google_ads_campaign_crm_performancepostfix': [
        'CAC',
        'campaign_id',
        'campaign_name',
        'customer_id',
        'new_customers',
        'segments_date',
        'spend'
    ],
    'google_ads_ad_group_detailspostfix': [
        'ad_group_id',
        'ad_group_name',
        'cost_per_conversion',
        'customer_id',
        'metrics_conversions',
        'metrics_cost_micros',
        'segments_date'
    ],
    'google_ads_ad_group_performancepostfix': [
        'ad_group_id',
        'ad_group_name',
        'cost_per_conversion',
        'customer_id',
        'metrics_conversions',
        'metrics_cost_micros',
        'segments_date'
    ]
}

────────────────────────────────────────────── 💡 Retrieved Examples ──────────────────────────────────────────────

Example #1

Query: Over the past 14 days - Which campaign had the lowest conversion rate?

...

Query: Which campaign had the maximum impressions on 1 jan 2025?

SQL: SELECT gcd.campaign_id, gcd.campaign_name, SUM(gcp.metrics_impressions) AS total_impressions FROM 
projectx-pipeline.dataset.google_ads_campaign_performancepostfix gcp JOIN 
projectx-pipeline.dataset.google_ads_campaign_detailspostfix gcd ON gcp.campaign_id = gcd.campaign_id WHERE 
gcp.customer_id = 8870377244 AND gcp.segments_date = '2025-01-01' GROUP BY gcd.campaign_id, gcd.campaign_name ORDER
BY total_impressions DESC LIMIT 1

Example #5

Query: Which ad group had the maximum cost per conversion on 2 June 2025 ?

SQL: SELECT gag.ad_group_name, 
       SAFE_DIVIDE(SAFE_DIVIDE(SUM(ga.metrics_cost_micros), 1000000), SUM(ga.metrics_conversions)) AS 
cost_per_conversion
FROM projectx-pipeline.dataset.google_ads_ad_group_performancepostfix ga
JOIN projectx-pipeline.dataset.google_ads_ad_group_detailspostfix gag ON ga.ad_group_id = gag.ad_group_id
WHERE ga.customer_id = 3424742100
AND ga.segments_date = '2025-06-02'
GROUP BY gag.ad_group_name
ORDER BY cost_per_conversion DESC
LIMIT 1